In [17]:
!pip install statsmodels
!pip install pyspark
!pip install numpy

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [6]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql import functions as F
import matplotlib.pyplot as plt
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from pyspark.sql.functions import col
import numpy as np
from datetime import datetime, timedelta

In [7]:
import findspark
findspark.init()
import pyspark

sc = pyspark.SparkContext(master='spark://hd-master:7077',
                          appName='big_data')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/30 11:34:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
spark = SQLContext(sc)

/home/hadoop/.local/lib/python3.10/site-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [11]:
df = spark.read.option("header",True).csv("hdfs://hd-master:9000/covid-19", inferSchema=True)

In [12]:
df.show(5,truncate=False)

+-----------+----------+---------+---------+------+
|country    |date      |confirmed|recovered|deaths|
+-----------+----------+---------+---------+------+
|afghanistan|2020-01-22|0        |0        |0     |
|afghanistan|2020-01-23|0        |0        |0     |
|afghanistan|2020-01-24|0        |0        |0     |
|afghanistan|2020-01-25|0        |0        |0     |
|afghanistan|2020-01-26|0        |0        |0     |
+-----------+----------+---------+---------+------+
only showing top 5 rows



In [13]:
# Global Parameters
time_seires_num = 30
country_you_want_to_predict = "china"
case_you_want_to_predict = "confirmed"
prediction_date = "2022-09-01" # The date should not exceed 2023-03-10

In [14]:
df = df.filter(df['country']==country_you_want_to_predict).select("date", case_you_want_to_predict)
df = df.withColumnRenamed(case_you_want_to_predict, "cases")

In [18]:
df.show(5,truncate=False)

+----------+-----+
|date      |cases|
+----------+-----+
|2020-01-22|548  |
|2020-01-23|643  |
|2020-01-24|920  |
|2020-01-25|1406 |
|2020-01-26|2075 |
+----------+-----+
only showing top 5 rows



In [24]:
def predict_time_series_arima(prediction_date):
    data = df
    # Calculate the difference
    prediction_date_dt = datetime.strptime(prediction_date, "%Y-%m-%d")
    first_date_in_training_set = prediction_date_dt - timedelta(days=time_seires_num)
    last_date_in_training_set = prediction_date_dt - timedelta(days=1)

    # Get the true data
    true_value = data.filter(col("date") == prediction_date).select("cases").collect()[0]["cases"]
    
    # Filter the date in data
    data = data.filter((col("date")>=first_date_in_training_set) & (col("date")<=last_date_in_training_set))
    
    # Assemble features
    assembler = VectorAssembler(inputCols=["cases"], outputCol="features")
    assembled_data = assembler.transform(data)

    # Extract features as a NumPy array
    np_data = np.array(assembled_data.select("features").rdd.map(lambda x: x[0].toArray()[0]).collect())

    # Fit an ARIMA model
    order = (1, 1, 1)  # Example order, you may need to tune this based on your data
    model = ARIMA(np_data, order=order)
    fit_model = model.fit()

    prediction_value = int(fit_model.forecast(1)[0])
    error = abs(prediction_value - true_value)/true_value*100
    print(f"Predicted value for {prediction_date}: {prediction_value}; True value for {prediction_date}: {true_value}; The error is: {error}%")
    return error

In [25]:
predict_time_series_arima(prediction_date)

23/11/30 11:39:33 WARN TaskSetManager: Lost task 0.0 in stage 11.0 (TID 28) (172.20.10.5 executor 1): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 152, in load_stream
    yield self._read_with_length(stream)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 174, in _read_with_length
    return self.loads(obj)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 472, in loads
    return cloudpickle.loads(obj, encoding=encoding)
  File "/usr/local/

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 11.0 failed 4 times, most recent failure: Lost task 0.3 in stage 11.0 (TID 32) (172.20.10.5 executor 1): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 152, in load_stream
    yield self._read_with_length(stream)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 174, in _read_with_length

23/11/30 11:44:32 ERROR TaskSchedulerImpl: Lost executor 0 on 172.20.10.13: worker lost: Not receiving heartbeat for 60 seconds
----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 41958)
Traceback (most recent call last):
  File "/usr/lib/python3.10/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.10/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.10/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.10/socketserver.py", line 747, in __init__
    self.handle()
  File "/home/hadoop/.local/lib/python3.10/site-packages/pyspark/accumulators.py", line 295, in handle
    poll(accum_updates)
  File "/home/hadoop/.local/lib/python3.10/site-packages/pyspark/accumulators.py", line 267, in poll
    if self.rfile